<a href="https://colab.research.google.com/github/Raian-Rahman/DhakaAI2020/blob/main/Inference_Notebook_Team_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains only the inference from our trained weight by Team_Perceptron. Here are some insights that we got from our training. 

- There were some inconsistency about the training dataset and on second round we corrected many of the labels. We also added some of our captured images in training. 
- In training, we used yolov5 implementaion by **ultralytics**. As the github repository changes by a lot margin, we downloaded a zip version of the implementation and added it from our drive. 

#  **Uploading the hidden Test File**

The hidden test (complete test data) should be uploaded into the following directory.
`
/content/yolov5/data/hidden_test/
`
The inference code will test the image from hidden_test folder. 

**Only the images should be in the hidden_test folder. The organizer can either include it from their drive or manually upload it**

#1. Setup 
In this step the system is setted up for inference. As we stated already, the source code will be downloaded from our drive. The following block does the job. 

**Here the model name is folded model (we saved it as our implementation)**

In [1]:
from IPython.display import clear_output
!rm -r sample_data
!rm -r /content/content/
!rm -r /content//yolov5

!gdown --id '1PT9iaMtxHTOcI1LGvNdfda-rVSLk5CNj'
!unzip FoldedModel.zip; rm FoldedModel.zip;
clear_output()

In [2]:
%cd /content/yolov5
!pip install -qr requirements.txt

/content/yolov5
     |████████████████████████████████| 645kB 17.0MB/s 


The following block creates a folder in data directory for hidden_test

In [3]:
%cd /content/yolov5/data/
!mkdir hidden_test

/content/yolov5/data


# 2. Downloading Your Processed Data
Here we have included a dataset from which we have trained our model. As we used multiple folding technique, the name of our dataset is fold0,fold1,...,foldN. The main difference between these dataset is the train and validation set. 

Initially we preprocessed the dataset annotation into **.txt format** and then we folded the dataset by splitting it. 

The following block downloads the preprocessed dataset. 


(Metadata is not needed for train in yolov5 but we included it)

In [4]:
%cd /content/yolov5/data/

# !gdown --id 'ID of your preprocessed .zip file'
!gdown --id '1U64EuZ1ZghBpue63BN0RoBKnmz4ztTe4'
!unzip Fold0.zip; rm Fold0.zip;
clear_output()

# Setting the metadata files in their perfect place
%cp '/content/yolov5/data/metadata/train.txt' /content/yolov5/data/
%cp '/content/yolov5/data/metadata/valid.txt' /content/yolov5/data/
%cp '/content/yolov5/data/metadata/test.txt' /content/yolov5/data/
%cp '/content/yolov5/data/metadata/traffic.names' /content/yolov5/data/
%cp '/content/yolov5/data/metadata/traffic.data' /content/yolov5/data/

# 3. Uploading the hidden Test File

The hidden test (complete test data) should be uploaded into the following directory.
`
/content/yolov5/data/hidden_test/
`
The inference code will test the image from hidden_test folder. 

**Only the images should be in the hidden_test folder. The organizer can either include it from their drive or manually upload it**

# 4. Donwloading Trained Weights: 

After trained weights are downloaded from our drive to the following directory. 
`
/content/yolov5/weights/
`
The following block downloads the weights from our drive. 

In [5]:
%cd /content/yolov5/weights/

!gdown --id '1--wPks51iiimZdwmf2a0f1e-FvAxDUHm' #res 640
!gdown --id '1-ZH_kdA_nZYmI9XiVqCTdsFkXVvkBSIq' #res 1024 fold 1
!gdown --id '1-40w_EJTondyT9aD0PPJAmgP1A4ROkEe' #res 1024 fold 2
!gdown --id '187eJgVdAKcMeO-G5KkLOtynryV65h-Wq' #res 1024 complete corrected dataset

/content/yolov5/weights
Downloading...
From: https://drive.google.com/uc?id=1--wPks51iiimZdwmf2a0f1e-FvAxDUHm
To: /content/yolov5/weights/w4.pt
178MB [00:02, 78.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-ZH_kdA_nZYmI9XiVqCTdsFkXVvkBSIq
To: /content/yolov5/weights/w3.pt
710MB [00:15, 45.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-40w_EJTondyT9aD0PPJAmgP1A4ROkEe
To: /content/yolov5/weights/w2.pt
710MB [00:17, 40.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=187eJgVdAKcMeO-G5KkLOtynryV65h-Wq
To: /content/yolov5/weights/w1.pt
710MB [02:30, 2.19MB/s]


# 5. Submission

This part of the code deals with making the submission file. Remember, the actual yolo v5 model outpus in the format  `(object-id) (score) (x-centre) (y-centre) (width) (height)` in each line for each object in the text label file. However, in the competition, you need to submit in this format `(image_name) (object_name) (xmin) (ymin) (xmax) (ymax) (width) (height)` for each line for each object in a `.csv` file.

So, first the inference is run and then the CSV is generated. **CSV is stored inside the yolov5 folded along with the timestamp**


In [ ]:
%cd /content/yolov5/
from models import *  
from utils.datasets import *
from utils.utils import *
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords,
    xyxy2xywh, strip_optimizer, set_logging)
from utils.torch_utils import select_device, load_classifier, time_synchronized

def detect(save_img=False):
    imgsz = opt.img_size 
    out, source, weights, half, view_img, save_txt = opt.output, opt.source, opt.weights, opt.half, opt.view_img, opt.save_txt

    # Initialize
    device = torch_utils.select_device(opt.device)
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder

    # Initialize model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

    
    
    # Eval mode
    model.to(device).eval()

    # Half precision
    half = half and device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()

    # Set Dataloader
    vid_path, vid_writer = None, None
    save_img = True
    dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = load_classes(opt.names)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img.float()) if device.type != 'cpu' else None  # run once

    results=[]
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=opt.augment)[0]
        t2 = torch_utils.time_synchronized()

        # to float
        if half:
            pred = pred.float()

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        
        # Process detections
        for i, det in enumerate(pred):  # detections for image i
            
            p, s, im0 = path, '', im0s          

            save_path = str(Path(out) / Path(p).name)
            #print(p)
            s += '%gx%g ' % img.shape[2:]  # print string
            #print(s)
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from imgsz to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                xmin = []
                ymin = []
                xmax = []
                ymax = []
                scores = []
                labels_value=[]
                image_ids=[]
                # Write results
                for *xyxy, conf, cls in det:
                    if save_txt:  # Write to file
                        
                        conf_score = '%.2f' % (conf)
                        label_with_cls = '%s' % (names[int(cls)])
                        
                        labels_value.append(label_with_cls)
                        
                        xmin.append(int(xyxy[0]))
                        ymin.append(int(xyxy[1]))
                        xmax.append(int(xyxy[2]))
                        ymax.append(int(xyxy[3]))
                        
                        scores.append(conf_score)
                        image_ids.append(save_path)

                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
        result = {
            'image_id': image_ids,
            'score': scores,
            'class': labels_value,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax

            }

        results.append(result)
    if save_txt or save_img:
        print('Results saved to %s' % os.getcwd() + os.sep + out)

    print('Done. (%.3fs)' % (time.time() - t0))
    return results

/content/yolov5


The hyperparameters for inference is setted in this class. **PLEASE DON'T CHANGE ANYTHING ON THE FOLLOWING BLOCK**

In [ ]:
# Setting up parameters for inference /submission
class opt:
    cfg='/content/yolov5/models/yolov5x.yaml'
    names='/content/yolov5/data/traffic.names'
    weights= ['/content/yolov5/weights/w1.pt', 
              '/content/yolov5/weights/w2.pt', 
              '/content/yolov5/weights/w3.pt', 
              '/content/yolov5/weights/w4.pt']
    source='/content/yolov5/data/hidden_test' #directory to the hidden test data
    save_txt=True
    output='output'  # Output directory of the results
    classes=False
    img_size=1024    # Inference Imag Size
    conf_thres=0.25
    iou_thres=0.55
    fourcc='mp4v'
    half=False
    device=''
    view_img=False
    agnostic_nms=False
    augment=False

In [ ]:
# predict results
with torch.no_grad():
    res=detect()

In the submission file you need to have to following headers with values



*   `image_id:` image file name
*   `class:` object name like `rickshaw` or `bus`
*   `score:` confidence score of the object detection
*   `xmin:` 
*   `ymin`
*   `xmax`
*   `ymax`
*   `height`
*   `width`

**As we resized all the test images into 1024 by 1024, so the output is scaled by 1024 by 1024**




The following block generates the csv file which will be saved in the `/content/yolov5/` directory. 

In [ ]:
import pandas as pd
import numpy as np

append_data=[]

for i in range(len(res)):
  df = pd.DataFrame(res[i], columns = ['image_id','class','score','xmin','ymin','xmax','ymax'])
  append_data.append(df)

finl_results=pd.concat(append_data)
finl_results.image_id = [x.strip('output/') for x in finl_results.image_id]

finl_results['width'] = 1024
finl_results['height'] = 1024

print(f'Number of Objects Detected: {finl_results.shape[0]}')

# Saving the submission file with timestamp
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

finl_results.to_csv(f'Team_Perceprtron {timestamp} .csv', index=False)
print('Submission file is written Successfully!\n\n Sneak Peak at Submission File :)')

finl_results.head()